# Preprocesado de datasets

En este proyecto, trabajamos con 6 datasets de información de e-commerce en Brasil, proporcionados por Olist.

In [17]:
from pathlib import Path

import pandas as pd
import numpy as np

In [2]:
BASE_DIR = Path.cwd().parent
DATA_DIR = (BASE_DIR / "data").resolve()

In [3]:
customers = pd.read_csv(DATA_DIR/"raw"/"olist_customers_dataset.csv")
df_customers = pd.DataFrame(customers)

items = pd.read_csv(DATA_DIR/"raw"/"olist_order_items_dataset.csv")
df_items = pd.DataFrame(items)

payments = pd.read_csv(DATA_DIR/"raw"/"olist_order_payments_dataset.csv")
df_payments = pd.DataFrame(payments)

reviews = pd.read_csv(DATA_DIR/"raw"/"olist_order_reviews_dataset.csv")
df_reviews = pd.DataFrame(reviews)

orders = pd.read_csv(DATA_DIR/"raw"/"olist_orders_dataset.csv")
df_orders = pd.DataFrame(orders)

products = pd.read_csv(DATA_DIR/"raw"/"olist_products_dataset.csv")
df_products = pd.DataFrame(products)

In [4]:
print("\ncustomer columns:\n", df_customers.columns)
print("\nitems columns:\n", df_items.columns)
print("\npayments columns:\n", df_payments.columns)
print("\nreviews columns:\n", df_reviews.columns)
print("\norders columns:\n", df_orders.columns)
print("\nproducts columns:\n", df_products.columns)


customer columns:
 Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

items columns:
 Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

payments columns:
 Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')

reviews columns:
 Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

orders columns:
 Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

products columns:
 Index(['product_id', 'product_category_name', 'product_name

Podemos observar que no todos los DataFrames tienen las mismas columnas, por lo que tendremos que hacer varios merges entre ellas.

Primero comenzare por los que puedo unir a traves de la columna *"order_id"*

In [5]:
print(df_items['order_id'].value_counts().max())
print(df_payments['order_id'].value_counts().max())
print(df_reviews['order_id'].value_counts().max())
print(df_orders['order_id'].value_counts().max())

21
29
3
1


Estas columnas tienen duplicados menos en el dataframe *df_orders*, por lo tanto habrá que tratar esto antes de realizar el join

### Payments Dataset

In [6]:
# Con esto conseguimos evitar duplicados al hacer merge de datasets por tener order_id duplicados

payments_total = df_payments.groupby("order_id", as_index=False)["payment_value"].sum()
payments_total.rename(columns={"payment_value": "total_payment"}, inplace=True)

payment_type_main = df_payments.groupby("order_id")["payment_type"].agg(lambda x: x.mode()[0]).reset_index()

max_installments = df_payments.groupby("order_id", as_index=False)["payment_installments"].max()
max_installments.rename(columns={"payment_installments": "max_installments"}, inplace=True)

df_payments = payments_total.merge(payment_type_main, on="order_id", how="left")
df_payments = df_payments.merge(max_installments, on="order_id", how="left")

df_payments.head(3)

,order_id,total_payment,payment_type,max_installments
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,credit_card,2
1,00018f77f2f0320c557190d7a144bdd3,259.83,credit_card,3
2,000229ec398224ef6ca0657da4fc703e,216.87,credit_card,5


Con esto, el dataset *"payments"* tiene order id individuales, de manera que no se harán combinaciones al hacer merge con las demas tablas por producto cartesiano.

### Items dataset

In [7]:
df_items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


Lo primero al observar este dataset es que hay columnas que descartar, por no aportar a nuestra hipotesis de predecir churn. Estas son:
- "seller_id"
- "product_id"
- "shipping_limit_date"

*"shipping_limit_date"* la eliminaremos dado que ya tenemos la fecha estimada de entrega y la fecha de entrega en el dataset

Además crearemos una nueva columna, *"item_count"* mostrando el numero total de items por order

In [8]:
total_price = df_items.groupby("order_id", as_index=False)["price"].sum()
total_price.rename(columns={"price": "total_price"}, inplace=True)

total_items = df_items.groupby("order_id").size().reset_index(name="item_count")
total_items

freight_price = df_items.groupby("order_id", as_index=False)["freight_value"].sum()
freight_price.rename(columns={"freight_value": "total_freight_value"}, inplace=True)

df_items = total_price.merge(total_items, on="order_id", how="left")
df_items = df_items.merge(freight_price, on="order_id", how="left")

df_items

,order_id,total_price,item_count,total_freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,1,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,1,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,1,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,1,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,1,18.14
...,...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,299.99,1,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,350.00,1,36.53
98663,fffce4705a9662cd70adb13d4a31832d,99.90,1,16.95
98664,fffe18544ffabc95dfada21779c9644f,55.99,1,8.72


Con esto he conseguido el total gastado por order, la cantidad de articulos y el total de coste por transporte, más adelante esto puede servirnos para hacer ingenieria de características

### Reviews dataset

Este es uno de los más importantes, ya que una mala review puede ser un indicativo de que alguien va a dejar el servicio. Lo primero que nos interesa es saber que tipos de datos tenemos por columna.

In [9]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [10]:
df_reviews.sample(3)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
85978,0c39a257fd58b5df6440064ef5670ac7,599e2a8d1c0c82f50ac6d898c2efaa94,4,NaN,Muito ruim a entrega do correios... No site co...,2018-01-05 00:00:00,2018-01-05 19:45:49
48562,5c21268a0f8bae726edb3aa4db3e6bde,0036757472ece3dde52fd4bfd929c90e,2,Mesa com cavaletes,"Demora absurda para entrega, dentro do prazo e...",2018-08-10 00:00:00,2018-08-14 02:57:36
66101,a53877b8ed65424a48dc80692b074cad,a74773544b428f3d89597084120ec144,4,NaN,NaN,2018-06-05 00:00:00,2018-06-07 19:33:46


Con esto podemos ver que tenemos una columna con el score de la review, que nos va a dar mucha información, sumado a el comentario de esta misma review.

Las fechas es algo que podremos descartar al final, ya que no nos proporciona información, junto con el review id.

Algo a tener en cuenta en el EDA es la cantidad de valores nulos en el titulo de las reviews y en las propias reviews.

In [11]:
# Comprobamos si hay orders duplicados
df_reviews["order_id"].value_counts().max()

np.int64(3)

Hay orders duplicados, esto puede significar que una persona editó su review para cambiar su nota.

Esto nos puede proporcionar información valiosa, podemos saber si la nota cambió, cuántas reviews totales puso un cliente por order y sacar una media de las notas.

En cuanto al texto de la review, nos quedaremos con el más reciente, en caso de querer utilizarlo más adelante.

Para comenzar, ordenaremos el dataset por orden cronológico, despues crearemos las columnas que nos pueden proporcionar más datos y despues haremos el merge

In [12]:
df_reviews["review_creation_date"] = pd.to_datetime(df_reviews["review_creation_date"])
df_reviews.sort_values("review_creation_date", inplace=True)

# Dejo esto aquí porque es mi primera vez usando .agg() así no me pierdo.

# df_total_rewiews = ( 
#     df_reviews
#         .groupby("order_id", as_index=False)["order_id"]
#         .count()
#         .rename(columns={"order_id": "total_reviews"})
# )

# df_mean_reviews = (
#     df_reviews
#         .groupby("order_id", as_index=False)["review_score"].mean()
#         .rename(columns={"review_score": "mean_score"})
# )

# df_worst_score = (
#     df_reviews
#     .groupby("order_id", as_index=False)["review_score"].min()
#     .rename(columns={"review_score": "min_score"})
# )

# df_last_score = (
#     df_reviews
#       .drop_duplicates(subset="order_id", keep="last")   # la última tras el sort
#       .loc[:, ["order_id", "review_score"]]
#       .rename(columns={"review_score": "last_score"})
# )

# Optimización de codigo usando función .agg()
agg_reviews = (df_reviews
                .groupby("order_id")
                .agg(
                    total_reviews = ("order_id", "size"),
                    worst_review = ("review_score", "min"),
                    mean_review_score = ("review_score", "mean"),
                    last_review = ("review_score", "last")
                )

)

agg_reviews

,total_reviews,worst_review,mean_review_score,last_review
order_id,,,,
00010242fe8c5a6d1ba2dd792cb16214,1,5,5.0,5
00018f77f2f0320c557190d7a144bdd3,1,4,4.0,4
000229ec398224ef6ca0657da4fc703e,1,5,5.0,5
00024acbcdf0a6daa1e931b038114c75,1,4,4.0,4
00042b26cf59d7ce69dfabb4e55b4fd9,1,5,5.0,5
...,...,...,...,...
fffc94f6ce00a00581880bf54a75a037,1,5,5.0,5
fffcd46ef2263f404302a634eb57f7eb,1,5,5.0,5
fffce4705a9662cd70adb13d4a31832d,1,5,5.0,5


In [13]:
df_reviews.drop(columns=["review_id", "review_creation_date", "review_answer_timestamp"], inplace=True)
df_reviews.head()

,order_id,review_score,review_comment_title,review_comment_message
70906,809a282bbd5dbcabb6f2f724fca862ec,1,NaN,MEU PEDIDO NÃO FOI ENTREGUE E NÃO FOI DADA NEN...
37547,bfbd0f9bdef84302105ad712db648a6c,1,NaN,nao recebi o produto e nem resposta da empresa
5503,e5215415bb6f76fe3b7cb68103a0d1c0,1,NaN,"PRODUTO NÃO CHEGOU,E JÁ PASSOU O PRAZO DE ENTREGA"
60439,e2144124f98f3bf46939bc5183104041,4,NaN,NaN
41042,9aa3197e4887919fde0307fc23601d7a,4,NaN,Só chegou uma parte do pedido ate agora..


### Order Dataset

Por último, el dataset de orders, tiene valores individuales para cada order, por lo tanto no hace falta procesarlo

In [14]:
df_orders["order_id"].value_counts().max()

np.int64(1)

## Union de datasets order_id

In [15]:
# Lo primero es comprobar si todos tienen las mismas dimensiones

print(df_items.shape)
print(df_orders.shape)
print(df_payments.shape)
print(df_reviews.shape)

(98666, 4)
(99441, 8)
(99440, 4)
(99224, 4)


In [16]:
for nombre, df in [("items", df_items),
                   ("payments", df_payments),
                   ("reviews", df_reviews),
                   ("orders", df_orders)]:
    dups = df.duplicated("order_id").sum()  
    print(f"{nombre:<8}  filas = {len(df):>6,}   pedidos únicos = {df['order_id'].nunique():>6,}   duplicados = {dups}")

items     filas = 98,666   pedidos únicos = 98,666   duplicados = 0
payments  filas = 99,440   pedidos únicos = 99,440   duplicados = 0
reviews   filas = 99,224   pedidos únicos = 98,673   duplicados = 551
orders    filas = 99,441   pedidos únicos = 99,441   duplicados = 0
